In [60]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

In [ ]:
def parse_html(html_content):

    """
    Parse the HTML content to extract movie information from a table.

    Parameters:
    - html_content (str): The HTML content of the webpage.

    Returns:
    - pd.DataFrame: A DataFrame containing the extracted movie information.
    """

    #create an emoty list to store all the results
    movies = []

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table containing the movie information
    table = soup.find('table')
    
    # Check if a table is found
    if table is not None:

        # Extract the header row
        header_row = table.find('tr')
        
        # Check if the header row is found
        if header_row is not None:
            
            # Get the index of each column
            columns = [col.get_text(strip=True) for col in header_row.find_all('th')]

            # Extract information for each movie
            for row in table.find_all('tr')[1:]:  # Skip the header row
                movie_info = [data.get_text(strip=True) for data in row.find_all('td')]
                movies.append(dict(zip(columns, movie_info)))

        else:
            print("No header row found in the table.")
    else:
        print("No table found on the webpage.")

    return pd.DataFrame(movies)

In [61]:
def get_html_content(url,headers,next_button_x_path):
    """
    Scrape HTML content from a webpage using Selenium.

    Parameters:
    - url (str): The URL of the webpage to scrape.
    - headers (dict): Headers to be included in the HTTP request.
    - next_button_x_path (str): XPath of the 'Load More' or next button.

    Returns:
    - pd.DataFrame: A DataFrame containing the scraped information.
    """

    # Set up the Safari WebDriver
    driver = webdriver.Safari()

    driver.get(url)
    df_list = []

    # Assuming there's a button to load more content, find and click it in a loop
    while True:
        try:
            # Get the current URL after clicking the button
            next_page_url = driver.current_url

            # Use the next_page_url to fetch HTML content
            html_content = requests.get(next_page_url, headers=headers).content

            df = parse_html(html_content)
            df_list.append(df)

            # Find and click the button to load more content
            load_more_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,next_button_x_path )))
            load_more_button.click()

            # Wait for some time to let the new content load
            time.sleep(1)  # Adjust this based on your website's loading time

        except Exception as e:
            # Break the loop if there is an exception or the button is not found
            print(f"An error occurred: {e}")
            break

    # Concatenate the list of DataFrames vertically
    result_df = pd.concat(df_list, axis=0, ignore_index=True)

    return result_df


In [62]:
# headers (dict): Headers to be included in the HTTP request.
headers = {'User-Agent': 'AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36'}

# url (str): The URL of the webpage to scrape.
url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/"

# next_button_x_path (str): XPath of the 'Load More' or next button.
next_button_x_path = '//*[@id="a-page"]/main/div/div/div[4]/ul/li[3]/a'

# get all the movie information
result_dataframe = get_html_content(url,headers,next_button_x_path)
result_dataframe

WebDriverException: Message: 


In [63]:
#Save movie
result_dataframe.to_csv('../external_dataset/boxofficemojo_dataframe.csv', index=False)